In [1]:
import pandas as pd
import torch
import networkx as nx
from matplotlib.pyplot import figure
from torch_geometric.data import Dataset, Data, DataLoader
from scipy.linalg import fractional_matrix_power
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import date
import time

In [2]:
path = '../blockchain_files/3000000to3999999_BlockTransaction/'

In [3]:
df = pd.read_csv(path+'processed.csv')

In [4]:
tx_counts = pd.read_csv(path+'tx_counts.csv')

In [5]:
tx_labels = tx_counts[['0','1']]

In [6]:
merged_df= df.merge(tx_labels,left_on='from', right_on='1',how='left')

In [7]:
merged_df.drop(columns=['1'],inplace=True)

In [8]:
merged_df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,0x56c830805669f366a7b93411588954e1e1b2aab6,4950000000000000000,39000,20000000000,21000,mining
1,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xde088812a9c5005b0dc8447b37193c9e8b67a1ff,1011962068786510200,90000,20000000000,21000,mining
2,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xdeb5ebe6676ddddb280ea44f284e100edb9b9d4b,1011957331582025000,90000,20000000000,21000,mining
3,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xe7268aadb21f48a3b65f0880b6b9480217995979,1008362508265353200,90000,20000000000,21000,mining
4,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xf8c911c68f6a6b912fe735bbd953c3379336cbf3,1016119717874404400,90000,20000000000,21000,mining


In [39]:
df['value'].astype(float)/(10**18)

0           4.950000
1           1.011962
2           1.011957
3           1.008363
4           1.016120
              ...   
12846966    0.050169
12846967    0.050144
12846968    0.050138
12846969    0.050136
12846970    0.050131
Name: value, Length: 12846971, dtype: float64

In [46]:
df['gasPrice'].astype(float)/(10**10)

0           2.0
1           2.0
2           2.0
3           2.0
4           2.0
           ... 
12846966    2.0
12846967    2.0
12846968    2.0
12846969    2.0
12846970    2.0
Name: gasPrice, Length: 12846971, dtype: float64

In [45]:
df['gasLimit']/10000

0           3.9
1           9.0
2           9.0
3           9.0
4           9.0
           ... 
12846966    9.0
12846967    9.0
12846968    9.0
12846969    9.0
12846970    9.0
Name: gasLimit, Length: 12846971, dtype: float64

In [48]:
df['gasUsed']/10000

0           2.1
1           2.1
2           2.1
3           2.1
4           2.1
           ... 
12846966    2.1
12846967    2.1
12846968    2.1
12846969    2.1
12846970    2.1
Name: gasUsed, Length: 12846971, dtype: float64

In [9]:
def receiving_transaction_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['to'] == node]
    
    neigh2 = timed_df[timed_df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = timed_df[timed_df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = timed_df[timed_df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [10]:
def send_extract_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['from'] == node]
    
    neigh2 = timed_df[timed_df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = timed_df[timed_df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = timed_df[timed_df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [11]:
st_time = 1484475035
duration = 700000
end_time = st_time+duration
addr = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
(neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,df)

In [12]:
neigh1.shape

(39427, 7)

In [14]:
neigh2

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed
7,1484475096,0xca3de95c2b3fcb03f0559c338bfea9cf843fcbbb,0x26054a924a3591f28bdd474521946fc19f5832b1,1036595029016571800,21000,20000000000,21000
23,1484475182,0x95c0663312c9357270039b8a827e7f4de46edbb9,0x91337a300e0361bddb2e377dd4e88ccb7796663d,999672640000000000,21000,20000000000,21000
33,1484475188,0xf8c911c68f6a6b912fe735bbd953c3379336cbf3,0x91337a300e0361bddb2e377dd4e88ccb7796663d,1015699720000000000,21000,20000000000,21000
36,1484475230,0x32f0bbcc1cf12b57afe38bfcd4599f786fa77d10,0x9c67e141c0472115aa1b98bd0088418be68fd249,93884158663403030,21147,61000000001,21000
38,1484475230,0x91ff48c7b28c14708f10405e1c5abc4eb417269c,0xebc7a4c0e432e9632fb1ff4d4e2d31055e0de75a,1001797325872912900,21000,20000000000,21000
...,...,...,...,...,...,...,...
229539,1485174547,0x7ad9956eeb0ffbfde42f9c31ae29041a1b89cf07,0x91337a300e0361bddb2e377dd4e88ccb7796663d,1013655850000000000,21000,20000000000,21000
229546,1485174613,0x24a071cad70a8cf8e5634b40e58daf30dc93111d,0x297bf553a281b59606b66460cea2157f97b1f335,2200000000000000000,21000,41000000000,21000
229556,1485174709,0xf0d97fb6e11fccdce1ade0b1b0a2964468895efb,0x91337a300e0361bddb2e377dd4e88ccb7796663d,1014108190000000000,21000,20000000000,21000
229595,1485174746,0x23b94ef1762445cd9b1813876145393ffeb06363,0x602441e9f0868fd9857b49b90455862e0cdb64fe,1000000000000000000,21000,41000000000,21000


In [13]:
nodes = pd.concat([df['from'],df['to']]).unique()
nodes.shape

(2639698,)

In [16]:
map_id[addr]

NameError: name 'map_id' is not defined

In [14]:
map_id = {j:i for i,j in enumerate(nodes)}

In [15]:
df_copy = df.copy()

In [16]:
merged_df['from'] = merged_df['from'].map(map_id)

In [17]:
merged_df['to'] = merged_df['to'].map(map_id)

In [18]:
merged_df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4950000000000000000,39000,20000000000,21000,mining
1,1484475035,1,1777,1011962068786510200,90000,20000000000,21000,mining
2,1484475035,1,20425,1011957331582025000,90000,20000000000,21000,mining
3,1484475035,1,1558,1008362508265353200,90000,20000000000,21000,mining
4,1484475035,1,15,1016119717874404400,90000,20000000000,21000,mining


In [19]:
merged_df['value'] = merged_df['value'].astype(float)/(10**18)

In [20]:
merged_df['gasPrice'] = merged_df['gasPrice'].astype(float)/(10**10)

In [21]:
merged_df['gasLimit'] = merged_df['gasLimit']/10000

In [22]:
merged_df['gasUsed'] = merged_df['gasUsed']/10000

In [23]:
merged_df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [18]:
!ls ../3000000to3999999_BlockTransaction

3000000to3999999_BlockTransaction.csv  processed.csv  time.csv
extracted_labels.csv		       readme.txt     tx_counts.csv


In [27]:
tx_counts = pd.read_csv(path+'tx_counts.csv')

In [28]:
tx_counts[tx_counts['tx_sent']>10]

,0,1,tx_sent,tx_rec
0,compromised,0x027beefcbad782faf69fad12dee97ed894c68549,11503.0,1160.0
1,compromised,0xb6aac3b56ff818496b747ea57fcbe42a9aae6218,11515.0,11573.0
2,compromised,0x0ee4e2d09aec35bdf08083b649033ac0a41aa75e,1919.0,33.0
3,exchange,0x1151314c646ce4e0efd76d1af4760ae66a9fe30f,49646.0,3.0
4,exchange,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,156152.0,371.0
5,exchange,0x007174732705604bbbf77038332dc52fd5a5000c,1187.0,3962.0
6,exchange,0xd7c866d0d536937bf9123e02f7c052446588189f,232.0,124.0
7,exchange,0x96fc4553a00c117c5b0bed950dd625d1c16dc894,46548.0,50585.0
8,exchange,0x4b01721f0244e7c5b5f63c20942850e447f5a5ee,846.0,652.0
9,exchange,0x167a9333bf582556f35bd4d16a7e80e191aa6476,39272.0,11.0


In [24]:
st_time = 1484475035
duration = 700000
end_time = st_time+duration
addr = 1
(neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,df_copy)

In [25]:
neigh2.shape

(0, 7)

In [33]:
print(date.fromtimestamp(st_time))
print(date.fromtimestamp(end_time))


2017-01-15
2017-01-23


In [34]:
df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed
0,1484475035,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,0x56c830805669f366a7b93411588954e1e1b2aab6,4950000000000000000,39000,20000000000,21000
1,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xde088812a9c5005b0dc8447b37193c9e8b67a1ff,1011962068786510200,90000,20000000000,21000
2,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xdeb5ebe6676ddddb280ea44f284e100edb9b9d4b,1011957331582025000,90000,20000000000,21000
3,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xe7268aadb21f48a3b65f0880b6b9480217995979,1008362508265353200,90000,20000000000,21000
4,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xf8c911c68f6a6b912fe735bbd953c3379336cbf3,1016119717874404400,90000,20000000000,21000


In [35]:
df['from'].isin(tx_counts['1'])

0           True
1           True
2           True
3           True
4           True
            ... 
12846966    True
12846967    True
12846968    True
12846969    True
12846970    True
Name: from, Length: 12846971, dtype: bool

In [52]:
tx_counts['1']

0     0xe76fe52a251c8f3a5dcd657e47a6c8d16fdf4bfa
1     0xa42af2c70d316684e57aefcc6e393fecb1c7e84e
2     0x2a65aca4d5fc5b5c859090a6c34d164135398226
3     0xa027231f42c80ca4125b5cb962a21cd4f812e88f
4     0xea674fdde714fd979de3edf0f56aa9716b898ec8
                         ...                    
74    0x9d551f41fed6fc27b719777c224dfecce170004d
75    0x6f931bac260e7fcbaa0244d3b43a2bd9e9acf698
76    0xba83e9ce38b10522e3d6061a12779b7526839eda
77    0xc57f1148855e67763a694f7f2c0e68230adc686e
78    0x001866ae5b3de6caa5a51543fd9fb64f524f5478
Name: 1, Length: 79, dtype: object

In [65]:
len(tx_counts[df['from'][0] == tx_counts['1']]['0'])

1

In [36]:
dfc = df.copy()

In [37]:
tx_labels = tx_counts[['0','1']]
tx_labels.head()

,0,1
0,compromised,0x027beefcbad782faf69fad12dee97ed894c68549
1,compromised,0xb6aac3b56ff818496b747ea57fcbe42a9aae6218
2,compromised,0x0ee4e2d09aec35bdf08083b649033ac0a41aa75e
3,exchange,0x1151314c646ce4e0efd76d1af4760ae66a9fe30f
4,exchange,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98


In [38]:
dfc.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed
0,1484475035,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,0x56c830805669f366a7b93411588954e1e1b2aab6,4950000000000000000,39000,20000000000,21000
1,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xde088812a9c5005b0dc8447b37193c9e8b67a1ff,1011962068786510200,90000,20000000000,21000
2,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xdeb5ebe6676ddddb280ea44f284e100edb9b9d4b,1011957331582025000,90000,20000000000,21000
3,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xe7268aadb21f48a3b65f0880b6b9480217995979,1008362508265353200,90000,20000000000,21000
4,1484475035,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0xf8c911c68f6a6b912fe735bbd953c3379336cbf3,1016119717874404400,90000,20000000000,21000


In [39]:
 merged_df= dfc.merge(tx_labels,left_on='from', right_on='1',how='left')

In [40]:
merged_df.shape

(12846971, 9)

In [41]:
df.shape

(12846971, 7)

In [26]:
merged_df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [27]:
merged_df['0'].unique()

array(['mining', nan, 'exchange', 'walletapp', 'compromised', 'gambling',
       'isowallet', 'phish_hack'], dtype=object)

In [47]:
labels[1]==None

False

In [28]:
labels = merged_df['0'].unique()
for label in labels:
    print('%s: '% label,(merged_df['0'] == label).sum())
# print('mining: ',(merged_df['0'] == 'mining').sum())

mining:  3499995
nan:  0
exchange:  1315278
walletapp:  52765
compromised:  24937
gambling:  823
isowallet:  135
phish_hack:  172


In [37]:
3499995 + 1315278 + 52765 + 24937 

4892975

In [29]:
merged_df.shape

(12846971, 8)

In [30]:
merged_df['0'].isnull().sum()

7952866

In [31]:
st_time = 1484475035
duration = 700000
end_time = st_time+duration
addr = 1
(neigh1,neigh2,neigh3,neigh4)=send_extract_graph_h4_timed(addr,st_time,end_time,merged_df)

In [32]:
neigh1.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining
18,1484475156,1,1890223,1.014285,9.0,2.0,2.1,mining


In [33]:
merged_df.shape

(12846971, 8)

In [34]:
merged_df.head()

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [35]:
path+'final.csv'

'../blockchain_files/3000000to3999999_BlockTransaction/final.csv'

In [36]:
!ls ../3000000to3999999_BlockTransaction/

ls: cannot access '../3000000to3999999_BlockTransaction/': No such file or directory


In [37]:
with open(path+'final.csv', 'w') as f:
        merged_df.to_csv(f,index=False,header=True)